In [1]:
import tensorflow as tf
print(tf.__version__)


2.17.0


In [ ]:
# i want to build the model to capture the  infant baby face expression and hand body movement to find the infant needs
# what i want to detect :-
# 1- baby face cry if he/she cry what is the reason behind it  and i have  cry folder inside the cry folder i have hungry,discomfort,pain folder and inside these folder having  image frames
# a-when baby is hungry baby hand will take inside the mouth,
# b- when baby discomfort he will lift by other person 
# c- when baby is in pain then face expression will be pain 

# can you suggest the roadmap

# combine model with audio and video
 


In [ ]:
# iss cobine model mei audio wala kaam ho rha hai pr video or pose nhi kam kr rha hai 

import os
import cv2
import dlib
import numpy as np
import librosa
import threading
import tensorflow as tf
from tensorflow.keras.models import load_model  # type: ignore
from joblib import load  # Import joblib to load .pkl files

# Load pre-trained models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r"D:\new_infant_project\3_final_model_combaine\shape_predictor_68_face_landmarks.dat")
emotion_model = load_model(r"D:\new_infant_project\3_final_model_combaine\emotion_detector.h5")
audio_model = load_model(r"D:\new_infant_project\3_final_model_combaine\best_baby_voice_model_yt_new_data.keras")
pose_model = load(r"D:\new_infant_project\3_final_model_combaine\pose_classifier.pkl")  # Load with joblib

# Define emotion and pose labels
emotion_labels_face = [
    'cry/discomfort', 'cry/hungry face',
    'happy/cooing', 'happy/laugh',
    'neutral/sleep', 'neutral/tired'
]
audio_labels = ['happy', 'cry', 'discomfort', 'neutral']
pose_labels = ['crawling', 'standing', 'sitting', 'prone', 'lying left', 'lying right', 'sleep']

# Function to extract audio from uploaded video
def extract_audio_from_video(video_path):
    import moviepy.editor as mp
    clip = mp.VideoFileClip(video_path)
    audio_path = "extracted_audio.wav"
    clip.audio.write_audiofile(audio_path)
    return audio_path

# Process audio and make predictions
def process_audio(audio_path):
    audio, sr = librosa.load(audio_path, sr=22050)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    mfcc = np.mean(mfcc.T, axis=0).reshape(1, -1, 1, 1)
    
    audio_prediction = audio_model.predict(mfcc)
    predicted_audio_emotion = audio_labels[np.argmax(audio_prediction)]
    return predicted_audio_emotion

# Function for pose detection based on pre-trained model
def detect_pose(frame):
    # Assuming `extract_keypoints` is defined for extracting pose keypoints
    keypoints = extract_keypoints(frame)
    keypoints = keypoints.reshape(1, -1)
    pose_prediction = pose_model.predict(keypoints)
    predicted_pose = pose_labels[pose_prediction[0]]
    return predicted_pose

# Process video frames and detect facial emotions and pose
def process_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    final_results = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)
        pose_prediction = detect_pose(frame)  # Call pose detection function

        for face in faces:
            x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
            face_region = gray[y1:y2, x1:x2]
            face_resized = cv2.resize(face_region, (48, 48)) / 255.0
            face_resized = face_resized.reshape(1, 48, 48, 1)

            emotion_prediction = emotion_model.predict(face_resized)
            predicted_emotion_face = emotion_labels_face[np.argmax(emotion_prediction)]
            
            # Aggregate results per frame
            final_results.append({
                "face_emotion": predicted_emotion_face,
                "pose": pose_prediction
            })

            # Visualize detection
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Emotion: {predicted_emotion_face}, Pose: {pose_prediction}",
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        
        cv2.imshow("Emotion and Pose Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    return final_results

# Aggregate model outputs and determine baby’s overall state
def determine_baby_state(audio_emotion, frame_results):
    baby_state = ""
    for result in frame_results:
        face_emotion = result['face_emotion']
        pose = result['pose']
        
        # Define rules for aggregating results
        if audio_emotion == "happy" and "happy" in face_emotion and pose == "standing":
            baby_state = "Baby is in a playful mood."
        elif audio_emotion == "happy" and "happy" in face_emotion and pose == "crawling":
            baby_state = "Baby is in a happy mood."
        elif audio_emotion == "neutral" and "sleep" in face_emotion and "lying" in pose:
            baby_state = "Baby is probably asleep."
        elif audio_emotion == "cry" and "hungry" in face_emotion and pose in ["lying left", "lying right"]:
            baby_state = "Baby is hungry and may need feeding."
        elif audio_emotion == "discomfort" and "cry" in face_emotion and pose == "prone":
            baby_state = "Baby might be in discomfort."
        else:
            baby_state = "Baby's state could not be determined."

        # Print current frame's results
        print(f"Audio Emotion: {audio_emotion}, Face Emotion: {face_emotion}, Pose: {pose}")
        print(f"Determined Baby State: {baby_state}")
    
    return baby_state

# Main function to upload video, process audio, and video
def main(video_path):
    # Step 1: Extract and analyze audio
    audio_path = extract_audio_from_video(video_path)
    audio_emotion = process_audio(audio_path)
    print(f"Predicted Audio Emotion: {audio_emotion}")
    
    # Step 2: Process video frames for face and pose detection
    frame_results = process_video_frames(video_path)
    
    # Step 3: Determine baby's overall state
    baby_state = determine_baby_state(audio_emotion, frame_results)
    print(f"Overall Baby State: {baby_state}")

# Run main with the path to the video
video_path = r'D:\new_infant_project\3_final_model_combaine\happy_sitting.mp4'
main(video_path)


MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 799ms/step
Predicted Audio Emotion: happy


NameError: name 'extract_keypoints' is not defined

In [ ]:
# eiss code mei sb working mei hai bss live capturing hai vdo ka
import os
import cv2
import dlib
import numpy as np
import librosa
import threading
import tensorflow as tf
from tensorflow.keras.models import load_model  # type: ignore
from joblib import load  # Import joblib to load .pkl files
import mediapipe as mp


# Load pre-trained models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r"D:\new_infant_project\3_final_model_combaine\shape_predictor_68_face_landmarks.dat")
emotion_model = load_model(r"D:\new_infant_project\3_final_model_combaine\emotion_detector.h5")
audio_model = load_model(r"D:\new_infant_project\3_final_model_combaine\best_baby_voice_model_yt_new_data.keras")
pose_model = load(r"D:\new_infant_project\3_final_model_combaine\pose_classifier.pkl")  # Load with joblib

# Define emotion and pose labels
emotion_labels_face = [
    'cry/discomfort', 'cry/hungry face',
    'happy/cooing', 'happy/laugh',
    'neutral/sleep', 'neutral/tired'
]
audio_labels = ['happy', 'cry', 'discomfort', 'neutral']
pose_labels = ['crawling', 'standing', 'sitting', 'prone', 'lying left', 'lying right', 'sleep']

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)

# Function to extract audio from uploaded video
def extract_audio_from_video(video_path):
    import moviepy.editor as mp
    clip = mp.VideoFileClip(video_path)
    audio_path = "extracted_audio.wav"
    clip.audio.write_audiofile(audio_path)
    return audio_path

# Process audio and make predictions
def process_audio(audio_path):
    audio, sr = librosa.load(audio_path, sr=22050)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    mfcc = np.mean(mfcc.T, axis=0).reshape(1, -1, 1, 1)
    
    audio_prediction = audio_model.predict(mfcc)
    predicted_audio_emotion = audio_labels[np.argmax(audio_prediction)]
    return predicted_audio_emotion

# Function to extract keypoints using MediaPipe
def extract_keypoints(image):
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.pose_landmarks:
        keypoints = np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.pose_landmarks.landmark]).flatten()
    else:
        keypoints = np.zeros(33 * 3)  # If no pose detected, return zeros
    return keypoints

# Process video frames and detect facial emotions and pose
def process_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    final_results = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        for face in faces:
            x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
            face_region = gray[y1:y2, x1:x2]
            face_resized = cv2.resize(face_region, (48, 48)) / 255.0
            face_resized = face_resized.reshape(1, 48, 48, 1)

            emotion_prediction = emotion_model.predict(face_resized)
            predicted_emotion_face = emotion_labels_face[np.argmax(emotion_prediction)]
            
            # Extract pose keypoints
            pose_keypoints = extract_keypoints(frame)
            pose_keypoints = pose_keypoints.reshape(1, -1)
            pose_prediction = pose_model.predict(pose_keypoints)
            predicted_pose = pose_labels[np.argmax(pose_prediction)]

            # Aggregate results per frame
            final_results.append({
                "face_emotion": predicted_emotion_face,
                "pose": predicted_pose
            })

            # Visualize detection
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Emotion: {predicted_emotion_face}, Pose: {predicted_pose}",
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        
        cv2.imshow("Emotion and Pose Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    return final_results

# Aggregate model outputs and determine baby’s overall state
def determine_baby_state(audio_emotion, frame_results):
    baby_state = ""
    for result in frame_results:
        face_emotion = result['face_emotion']
        pose = result['pose']
        
        # Define rules for aggregating results
        if audio_emotion == "happy" and "happy" in face_emotion and pose == "standing":
            baby_state = "Baby is in a playful mood."
        elif audio_emotion == "happy" and "happy" in face_emotion and pose == "crawling":
            baby_state = "Baby is in a happy mood."
        elif audio_emotion == "neutral" and "sleep" in face_emotion and "lying" in pose:
            baby_state = "Baby is probably asleep."
        elif audio_emotion == "cry" and "hungry" in face_emotion and pose in ["lying left", "lying right"]:
            baby_state = "Baby is hungry and may need feeding."
        elif audio_emotion == "discomfort" and "cry" in face_emotion and pose == "prone":
            baby_state = "Baby might be in discomfort."
        else:
            baby_state = "Baby's state could not be determined."

        # Print current frame's results
        print(f"Audio Emotion: {audio_emotion}, Face Emotion: {face_emotion}, Pose: {pose}")
        print(f"Determined Baby State: {baby_state}")
    
    return baby_state

# Main function to upload video, process audio, and video
def main(video_path):
    # Step 1: Extract and analyze audio
    audio_path = extract_audio_from_video(video_path)
    audio_emotion = process_audio(audio_path)
    print(f"Predicted Audio Emotion: {audio_emotion}")
    
    # Step 2: Process video frames for face and pose detection
    frame_results = process_video_frames(video_path)
    
    # Step 3: Determine baby's overall state
    baby_state = determine_baby_state(audio_emotion, frame_results)
    print(f"Overall Baby State: {baby_state}")

# Run main with the path to the video
video_path = r'D:\new_infant_project\3_final_model_combaine\happy_sitting.mp4'
main(video_path)


  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.


KeyboardInterrupt: 

In [ ]:
import os
import cv2
import dlib
import numpy as np
import librosa
from tensorflow.keras.models import load_model
from joblib import load
import mediapipe as mp
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext

# Load pre-trained models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r"D:\new_infant_project\3_final_model_combaine\shape_predictor_68_face_landmarks.dat")
emotion_model = load_model(r"D:\new_infant_project\3_final_model_combaine\emotion_detector.h5")
audio_model = load_model(r"D:\new_infant_project\3_final_model_combaine\best_baby_voice_model_yt_new_data.keras")
pose_model = load(r"D:\new_infant_project\3_final_model_combaine\pose_classifier.pkl")

# Define emotion and pose labels
emotion_labels_face = [
    'cry/discomfort', 'cry/hungry face',
    'happy/cooing', 'happy/laugh',
    'neutral/sleep', 'neutral/tired'
]
audio_labels = ['happy', 'cry', 'discomfort', 'neutral']
pose_labels = ['crawling', 'standing', 'sitting', 'prone', 'lying left', 'lying right', 'sleep']

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)

# Function to extract audio from uploaded video
def extract_audio_from_video(video_path):
    import moviepy.editor as mp
    clip = mp.VideoFileClip(video_path)
    audio_path = "extracted_audio.wav"
    clip.audio.write_audiofile(audio_path)
    return audio_path

# Process audio and make predictions
def process_audio(audio_path):
    audio, sr = librosa.load(audio_path, sr=22050)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    mfcc = np.mean(mfcc.T, axis=0).reshape(1, -1, 1, 1)
    
    audio_prediction = audio_model.predict(mfcc)
    predicted_audio_emotion = audio_labels[np.argmax(audio_prediction)]
    return predicted_audio_emotion

# Function to extract keypoints using MediaPipe
def extract_keypoints(image):
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.pose_landmarks:
        keypoints = np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.pose_landmarks.landmark]).flatten()
    else:
        keypoints = np.zeros(33 * 3)  # If no pose detected, return zeros
    return keypoints

# Process video frames and detect facial emotions and pose
def process_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    final_results = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        for face in faces:
            x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
            face_region = gray[y1:y2, x1:x2]
            face_resized = cv2.resize(face_region, (48, 48)) / 255.0
            face_resized = face_resized.reshape(1, 48, 48, 1)

            emotion_prediction = emotion_model.predict(face_resized)
            predicted_emotion_face = emotion_labels_face[np.argmax(emotion_prediction)]
            
            # Extract pose keypoints
            pose_keypoints = extract_keypoints(frame)
            pose_keypoints = pose_keypoints.reshape(1, -1)
            pose_prediction = pose_model.predict(pose_keypoints)
            predicted_pose = pose_labels[np.argmax(pose_prediction)]

            # Aggregate results per frame
            final_results.append({
                "face_emotion": predicted_emotion_face,
                "pose": predicted_pose
            })

    cap.release()
    return final_results

# Aggregate model outputs and determine baby’s overall state
def determine_baby_state(audio_emotion, frame_results):
    baby_state = ""
    for result in frame_results:
        face_emotion = result['face_emotion']
        pose = result['pose']
        
        # Define rules for aggregating results
        if audio_emotion == "happy" and "happy" in face_emotion and pose == "standing":
            baby_state = "Baby is in a playful mood."
        elif audio_emotion == "happy" and "happy" in face_emotion and pose == "crawling":
            baby_state = "Baby is in a happy mood."
        elif audio_emotion == "neutral" and "sleep" in face_emotion and "lying" in pose:
            baby_state = "Baby is probably asleep."
        elif audio_emotion == "cry" and "hungry" in face_emotion and pose in ["lying left", "lying right"]:
            baby_state = "Baby is hungry and may need feeding."
        elif audio_emotion == "discomfort" and "cry" in face_emotion and pose == "prone":
            baby_state = "Baby might be in discomfort."
        else:
            baby_state = "Baby's state could not be determined."
    
    return baby_state

# Main function to upload video, process audio, and video
def analyze_video(video_path):
    # Step 1: Extract and analyze audio
    audio_path = extract_audio_from_video(video_path)
    audio_emotion = process_audio(audio_path)

    # Step 2: Process video frames for face and pose detection
    frame_results = process_video_frames(video_path)

    # Step 3: Determine baby's overall state
    baby_state = determine_baby_state(audio_emotion, frame_results)
    return audio_emotion, frame_results, baby_state

# GUI application
def upload_video():
    video_path = filedialog.askopenfilename(filetypes=[("MP4 files", "*.mp4")])
    if video_path:
        try:
            audio_emotion, frame_results, baby_state = analyze_video(video_path)
            result_text = f"Predicted Audio Emotion: {audio_emotion}\n"
            result_text += f"Overall Baby State: {baby_state}\n\n"
            result_text += "Frame Results:\n"
            for frame in frame_results:
                result_text += f"Face Emotion: {frame['face_emotion']}, Pose: {frame['pose']}\n"
            result_display.delete(1.0, tk.END)
            result_display.insert(tk.END, result_text)
        except Exception as e:
            messagebox.showerror("Error", str(e))

# Set up the GUI
root = tk.Tk()
root.title("Baby Emotion and Pose Detection")
root.geometry("600x400")

upload_button = tk.Button(root, text="Upload Video", command=upload_video)
upload_button.pack(pady=20)

result_display = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=70, height=15)
result_display.pack(pady=20)

root.mainloop()


  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 538ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step


  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


...

  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '

